In [ ]:
!ls ../input/birdsong-recognition/train_audio

In [ ]:
from pathlib import Path
import numpy as np
import librosa
import IPython.display as ipd
from joblib import Parallel, delayed
from functools import partial
# import random
import tqdm
import tqdm.notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
paths = [p for p in Path('../input/birdsong-recognition/train_audio').iterdir()]

# Select All birds

In [ ]:
# random.seed(42)

In [ ]:
selected_birds = paths#random.sample(paths, 50)

In [ ]:
len(selected_birds)

# Convert audio to melspectogram and save

In [ ]:
sampling_rate = 16000
duration = 10

In [ ]:
def get_spectrum(sampling_rate, duration, bird_name, audio_path):
    try:
        data, _ = librosa.load(audio_path, sr=sampling_rate, duration=duration)
        data = np.pad(data, (0, sampling_rate * duration - data.shape[0]), mode='symmetric')
        spectogram = librosa.feature.melspectrogram(data.astype(np.float32), sr=sampling_rate, n_mels=128)
        name = bird_name + audio_path.name
        return (name, spectogram)
    except ZeroDivisionError:
        pass

In [ ]:
spectograms = []
for bird_path in tqdm.notebook.tqdm(selected_birds):
    bird_name = bird_path.name
    spectograms.extend(Parallel(n_jobs=4)(delayed(partial(get_spectrum, sampling_rate, duration, bird_name))(path) for path in bird_path.iterdir()))

In [ ]:
spectograms = dict(filter(bool, spectograms))

In [ ]:
np.savez_compressed('spectograms.npz', **spectograms)

In [ ]:
!ls -lh